In [7]:
import cobra
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from cobra.flux_analysis import variability

expression_data = pd.read_csv('e_coli_core_expression.csv', index_col=0)
expression_data.sort_values(by=' reaction activity [mmol/gDW/h] ', ascending=False, inplace=True)
# expression_data

In [8]:
model = cobra.io.load_json_model('e_coli_core.json')
# model

Observations:
1. There are 95 reactions but only 73 have data!

# Excercise 1

No code required so answer is in the README.md

# Excercise 2

In [9]:
reactionsWithData = set(expression_data.index) # List of reactions with expression data from csv
reactionsInModel = set([rxn.id for rxn in model.reactions]) # List of reactions in the model
reactionsWithoutData = reactionsInModel - reactionsWithData # Set difference to find reactions without data
print(f"Number of reactions in the model: {len(reactionsInModel)}")
print(f"Number of reactions with expression data: {len(reactionsWithData)}")
print(f"Number of reactions without expression data: {len(reactionsWithoutData)}")
print(f"Reactions with expression data: {reactionsWithData}")
print(f"Reactions in the model but not in expression data: {reactionsWithoutData}")

Number of reactions in the model: 95
Number of reactions with expression data: 73
Number of reactions without expression data: 22
Reactions with expression data: {'SUCDi', 'GND', 'PGI', 'MALt2_2', 'SUCCt2_2', 'SUCOAS', 'LDH_D', 'RPE', 'ICL', 'ALCD2x', 'PGL', 'ATPS4r', 'ACONTa', 'GAPD', 'FBP', 'PFL', 'PGM', 'AKGt2r', 'FUM', 'CO2t', 'PPCK', 'PGK', 'ICDHyr', 'PTAr', 'TKT1', 'PIt2r', 'ACt2r', 'PYRt2', 'NH4t', 'PFK', 'GLUN', 'G6PDH2r', 'GLUt2r', 'GLUDy', 'PDH', 'O2t', 'ENO', 'MDH', 'FBA', 'GLUSy', 'FORt2', 'PYK', 'H2Ot', 'GLNS', 'FUMt2_2', 'NADH16', 'TPI', 'ME1', 'THD2', 'CYTBD', 'ACALDt', 'FORt', 'ADK1', 'RPI', 'MALS', 'GLNabc', 'FRUpts2', 'ETOHt2r', 'NADTRHD', 'ACONTb', 'CS', 'TKT2', 'GLCpts', 'ACKr', 'PPS', 'D_LACt2', 'AKGDH', 'TALA', 'ME2', 'PPC', 'FRD7', 'ACALD', 'SUCCt3'}
Reactions in the model but not in expression data: {'EX_glu__L_e', 'EX_fum_e', 'EX_acald_e', 'EX_akg_e', 'EX_ac_e', 'EX_h_e', 'EX_succ_e', 'BIOMASS_Ecoli_core_w_GAM', 'EX_gln__L_e', 'EX_h2o_e', 'EX_pi_e', 'EX_fru_e',

In [10]:
# Create a dictionary to map reaction IDs to their expression values
expression_dict = expression_data[' reaction activity [mmol/gDW/h] '].to_dict()

# reversible & irreversible reactions among the reactions with data
reversible_reactions = [rxn for rxn in model.reactions if rxn.id in expression_dict and rxn.reversibility]
irreversible_reactions = [rxn for rxn in model.reactions if rxn.id in expression_dict and not rxn.reversibility]
len(reversible_reactions), len(irreversible_reactions)

# NOTICE THAT GLUCOSE EXCHANGE & ATPM ARE NOT IN THE EXPRESSION DATA!!! Also the lengths of lists match the expected values :D

(39, 34)

In [12]:
# 1. For reversible reactions with expression data, set lower and upper bounds to -value and +value usinf expression_dict!
for rxn in reversible_reactions:
    value = expression_dict[rxn.id]
    rxn.lower_bound = -value
    rxn.upper_bound = value
    

# 2. For irreversible reactions with expression data, set lower and upper bounds to 0 and value or -value and 0 depending on the original bounds
# (THIS IS TRICKY BECAUSE THE DIRECTIONS ARE NOT CONSISTENT)
for rxn in irreversible_reactions:
    value = expression_dict[rxn.id]
    if rxn.upper_bound > 0: 
        rxn.upper_bound = value
    elif rxn.lower_bound < 0: 
        rxn.lower_bound = -value
        
# 3. For reactions without data, leave bounds as is. No change required since its in the NO DATA REACTIONS list

# 4. For glucose exchange reaction, set bounds to -1000 and 1000
glucose_exchange = model.reactions.get_by_id('EX_glc__D_e')
glucose_exchange.lower_bound = -1000
glucose_exchange.upper_bound = 1000

# 5. For ATPM reaction, leave bounds as is. No change required since its in the NO DATA REACTIONS list

In [13]:
# print a table listing each reaction’s lower and upper flux bound after implementing the above
for rxn in model.reactions:
    print(f"{rxn.id}: Lower Bound = {rxn.lower_bound}, Upper Bound = {rxn.upper_bound}")

PFK: Lower Bound = 0.0, Upper Bound = 12.12
PFL: Lower Bound = 0.0, Upper Bound = 1.0
PGI: Lower Bound = -13.12, Upper Bound = 13.12
PGK: Lower Bound = -23.13, Upper Bound = 23.13
PGL: Lower Bound = 0.0, Upper Bound = 8.12
ACALD: Lower Bound = -1.16, Upper Bound = 1.16
AKGt2r: Lower Bound = -3.1, Upper Bound = 3.1
PGM: Lower Bound = -20.01, Upper Bound = 20.01
PIt2r: Lower Bound = -6.03, Upper Bound = 6.03
ALCD2x: Lower Bound = -9.01, Upper Bound = 9.01
ACALDt: Lower Bound = -2.29, Upper Bound = 2.29
ACKr: Lower Bound = -1.19, Upper Bound = 1.19
PPC: Lower Bound = 0.0, Upper Bound = 2.56
ACONTa: Lower Bound = -25.35, Upper Bound = 25.35
ACONTb: Lower Bound = -25.35, Upper Bound = 25.35
ATPM: Lower Bound = 8.39, Upper Bound = 1000.0
PPCK: Lower Bound = 0.0, Upper Bound = 25.23
ACt2r: Lower Bound = -3.23, Upper Bound = 3.23
PPS: Lower Bound = 0.0, Upper Bound = 2.5
ADK1: Lower Bound = -30.57, Upper Bound = 30.57
AKGDH: Lower Bound = 0.0, Upper Bound = 24.35
ATPS4r: Lower Bound = -60.5, U

# Excercise 3

In [22]:
# compute FVA for all reactions 

fva = variability.flux_variability_analysis(
    model=model,
    reaction_list=model.reactions,
    loopless=False,
    fraction_of_optimum=0.0 # we do not impose any minimum growth level
)

# print a table with the reaction id and the corresponding min and max flux
print ("FVA minimal and maximal fluxes")
print(fva)

FVA minimal and maximal fluxes
           minimum    maximum
PFK       0.334677  11.153556
PFL       0.000000   1.000000
PGI      -0.085323   9.020351
PGK     -17.596002  -0.879355
PGL       0.000000   4.522388
...            ...        ...
NADH16    1.831111  20.260000
NADTRHD   0.000000   1.260000
NH4t      0.000000   3.450000
O2t       0.500000  12.256000
PDH       0.000000  10.931107

[95 rows x 2 columns]


In [20]:
# identify reactions with expression imposed maximum bounds whose feasible FVA maximum flux is smaller than the imposed upper bound

restricted_reactions = []

for rxn in model.reactions:
    if rxn.id in expression_dict:  
        if rxn.id == "FORt":
            continue
     # only consider reactions where we imposed bounds from expression data
    if rxn.id in expression_dict: 
        # the bound we set in Exercise 2
        max_bound = rxn.upper_bound  
        max_fva = fva.loc[rxn.id, "maximum"]  
        # FVA finds a positive flux, but smaller than the imposed bound
        if (max_fva > 0) and (max_fva < max_bound):
            restricted_reactions.append(rxn.id)

print("Number of reactions with max FVA flux < upper bound:", len(restricted_reactions))
print("List of these reactions:", restricted_reactions)

Number of reactions with max FVA flux < upper bound: 36
List of these reactions: ['PFK', 'PGI', 'PGL', 'PIt2r', 'ACONTa', 'ACONTb', 'PPCK', 'ADK1', 'AKGDH', 'ATPS4r', 'PTAr', 'PYK', 'CO2t', 'RPE', 'CS', 'CYTBD', 'ENO', 'SUCDi', 'TALA', 'TKT1', 'TKT2', 'TPI', 'FBA', 'FUM', 'G6PDH2r', 'GAPD', 'GLCpts', 'GLUDy', 'GLUN', 'GLUSy', 'GND', 'ICDHyr', 'ICL', 'MALS', 'O2t', 'PDH']


In [19]:
# number of reactions that have a positive minimal flux 

# if the minimum value in FVA is >0 this reaction always carry flux in forward direction
positive_minimal_flux = fva[fva["minimum"] > 0]

print("Number of reactions with positive minimal flux:", len(positive_minimal_flux))
print("List of these reactions:", list(positive_minimal_flux.index))

Number of reactions with positive minimal flux: 16
List of these reactions: ['PFK', 'ATPM', 'ATPS4r', 'PTAr', 'CS', 'CYTBD', 'ENO', 'TPI', 'EX_h_e', 'EX_h2o_e', 'FBA', 'GAPD', 'GLCpts', 'ICDHyr', 'NADH16', 'O2t']


# Excercise 4